In [203]:
import pandas as pd
import numpy as np
from build_model import concat_earning
from copy import deepcopy

In [24]:
file_path = 'D:/projects/stock/intra_day/'
main_path = 'D:/projects/stock'
eod_path = 'D:/projects/stock/EOD_data/'

In [3]:
l = pd.read_csv('sp500_list.csv',index_col = 0)
etf_l = pd.read_csv('etf_sector_list.csv',index_col = 0).to_dict()['sector']

In [4]:
l['Sector'].unique()

array(['Industrials', 'Health Care', 'Information Technology',
       'Communication Services', 'Consumer Discretionary', 'Utilities',
       'Financials', 'Materials', 'Real Estate', 'Consumer Staples',
       'Energy'], dtype=object)

In [5]:
df = pd.read_csv(file_path+'XLC.csv',index_col = 0, parse_dates = True).sort_index(ascending = True)

In [12]:
sorted(l['Sub Industry'].unique())

['Advertising',
 'Aerospace & Defense',
 'Agricultural & Farm Machinery',
 'Agricultural Products',
 'Air Freight & Logistics',
 'Airlines',
 'Alternative Carriers',
 'Apparel Retail',
 'Apparel, Accessories & Luxury Goods',
 'Application Software',
 'Asset Management & Custody Banks',
 'Auto Parts & Equipment',
 'Automobile Manufacturers',
 'Automotive Retail',
 'Biotechnology',
 'Brewers',
 'Broadcasting',
 'Building Products',
 'Cable & Satellite',
 'Casinos & Gaming',
 'Commodity Chemicals',
 'Communications Equipment',
 'Computer & Electronics Retail',
 'Construction & Engineering',
 'Construction Machinery & Heavy Trucks',
 'Construction Materials',
 'Consumer Electronics',
 'Consumer Finance',
 'Copper',
 'Data Processing & Outsourced Services',
 'Department Stores',
 'Distillers & Vintners',
 'Distributors',
 'Diversified Banks',
 'Diversified Chemicals',
 'Diversified Support Services',
 'Drug Retail',
 'Electric Utilities',
 'Electrical Components & Equipment',
 'Electronic C

In [114]:
sub_dic = {}
for i in sorted(l['Sub Industry'].unique()):
    sub_dic[i] = l[l['Sub Industry'] == i]

In [152]:
sub_dic['Railroads']

,Security,Sector,Sub Industry,Headquarters Location,Date first added,CIK,Founded
Symbol,,,,,,,
CSX,CSX Corp.,Industrials,Railroads,"Jacksonville, Florida",9/30/1967,277948,1980
KSU,Kansas City Southern,Industrials,Railroads,"Kansas City, Missouri",5/24/2013,54480,NaN
NSC,Norfolk Southern Corp.,Industrials,Railroads,"Norfolk, Virginia",NaN,702165,1881/1894 (1980)
UNP,Union Pacific Corp,Industrials,Railroads,"Omaha, Nebraska",NaN,100885,1862


In [214]:
ret_dic = {k:{} for k in sub_dic}
for k in sub_dic:
    for i in l[l['Sub Industry'] == k].index:
        try:
            sec = pd.read_csv(eod_path+i+'.csv',index_col = 0, parse_dates = True).sort_index(ascending = True)['2020']
            ret_dic[k][i] = sec
        except:
            print(i)
            
def output_jump_intra(sub_group):
    df_l = []
    for k, v in ret_dic[sub_group].items():
        s = deepcopy(v)
        s['jump'] = s['open']/s['close'].shift(1) - 1
        s['intra'] = s['close']/s['open'] - 1
        s.columns = [k+'_'+x for x in s.columns]
        df_l.append(s)
    df = pd.concat(df_l,axis = 1)
    j = df[[x for x in df.columns if x.split('_')[-1] == 'jump']]
    intra = df[[x for x in df.columns if x.split('_')[-1] == 'intra']]
    return j, intra

BF.B


In [215]:
a,b = output_jump_intra('Oil & Gas Equipment & Services')

In [216]:
a

,BKR_jump,HAL_jump,NOV_jump,SLB_jump,FTI_jump
timestamp,,,,,
2020-01-02,NaN,NaN,NaN,NaN,NaN
2020-01-03,0.013276,0.024716,0.012272,0.020662,0.007501
2020-01-06,0.003120,0.009693,0.001972,0.001233,0.001854
2020-01-07,-0.010188,-0.001575,-0.006279,-0.007104,-0.011045
2020-01-08,-0.002790,-0.008107,-0.006002,-0.008126,-0.026403
...,...,...,...,...,...
2020-06-17,-0.003563,-0.005913,-0.003082,-0.002046,-0.016490
2020-06-18,-0.009938,-0.017747,-0.020408,-0.026223,-0.021250
2020-06-19,0.035382,0.044731,0.038305,0.037661,0.042146


In [217]:
b

,BKR_intra,HAL_intra,NOV_intra,SLB_intra,FTI_intra
timestamp,,,,,
2020-01-02,-0.002726,-0.000405,0.008383,-0.008638,-0.005594
2020-01-03,-0.011946,-0.020957,-0.008604,-0.010732,0.004188
2020-01-06,-0.007776,0.015600,0.003150,0.005171,0.005088
2020-01-07,-0.006730,-0.026824,-0.013033,0.001974,-0.013029
2020-01-08,-0.009992,-0.017164,-0.029388,-0.021599,-0.007748
...,...,...,...,...,...
2020-06-17,-0.040524,-0.036431,-0.053323,0.016402,-0.041916
2020-06-18,0.010665,0.036135,0.022500,0.045054,0.000000
2020-06-19,-0.071343,-0.053701,-0.078493,-0.058739,-0.073529


In [106]:
d = {}
for k,v in ret_dic.items():
    c = pd.DataFrame(v).corr()
    num = c.shape[0]
    ave_cor = c.values[np.triu_indices_from(c.values,1)].mean()
    d[k] = (num,ave_cor)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: Mean of empty slice.
  """


In [108]:
pd.DataFrame(d).transpose()

,0,1
Advertising,2.0,0.933046
Aerospace & Defense,10.0,0.636890
Agricultural & Farm Machinery,1.0,NaN
Agricultural Products,1.0,NaN
Air Freight & Logistics,4.0,0.634357
...,...,...
Tobacco,2.0,0.735877
Trading Companies & Distributors,1.0,NaN
Trucking,2.0,0.388077
Water Utilities,1.0,NaN


In [111]:
l

,Security,Sector,Sub Industry,Headquarters Location,Date first added,CIK,Founded
Symbol,,,,,,,
MMM,3M Company,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",8/9/1976,66740,1902
ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",3/31/1964,1800,1888
ABBV,AbbVie Inc.,Health Care,Pharmaceuticals,"North Chicago, Illinois",12/31/2012,1551152,2013 (1888)
ABMD,ABIOMED Inc,Health Care,Health Care Equipment,"Danvers, Massachusetts",5/31/2018,815094,1981
ACN,Accenture plc,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",7/6/2011,1467373,1989
...,...,...,...,...,...,...,...
YUM,Yum! Brands Inc,Consumer Discretionary,Restaurants,"Louisville, Kentucky",10/6/1997,1041061,NaN
ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",12/23/2019,877212,1969
ZBH,Zimmer Biomet Holdings,Health Care,Health Care Equipment,"Warsaw, Indiana",8/7/2001,1136869,NaN
